In [1]:
import pandas as pd
import numpy as np
import ast
from tqdm import tqdm_notebook as tqdm

In [2]:
data = pd.read_csv('data/rap_corpus.csv', sep=';')
data.shape

(5483, 2)

In [3]:
data.head()

,title,text
0,Многоточие (DOTSFAM) - Интро DOTS FAMILY,"['Dots Family!', 'Fuck you!', 'Dots Family!', ..."
1,Паук - Мой бит,"['Припев (4 раза):', 'Мой бит качает.', 'Он ни..."
2,Стриж - Алена,"['Ухоженная девочка', 'Брильянтовые стразы', '..."
3,Скато (Skato) - Хроника,"['Кровоточат раны, швы наложены на прошлое.', ..."
4,Bad Balance - Африка,['- Как уже сообщалось: в минувшую субботу поз...


In [4]:
data.tail()

,title,text
5478,"Dessar - Постой, не уходи","['Я тушу боль последнею сигаретой ', 'В глазах..."
5479,Big Black Boots - Нам хорошо (feat. Тэона),"['Вот и мы, а где были вы', 'Когда мы начинали..."
5480,Каста - Гончая,"['На спидометре сто двадцать- мчим,', 'Трек по..."
5481,Krec - В никуда,"['Взять чемодан и умчать в никуда, ', 'Может, ..."
5482,Schokk - Грустная песня (feat. Maxat),"['Я иногда, будто сквозь время вижу в степи ру..."


In [5]:
## преобразование текста в список
rapers = []
texts = []
for i in tqdm(range(data.shape[0])):
    tmp = data.iloc[i,:]
    rapers.append(tmp.title.split(' - ')[0])
    txt = ast.literal_eval(tmp.text)
    txt = [x.lower() for x in txt]
    texts.append(txt)

In [6]:
print(len(rapers))
print(len(texts))

5483
5483


In [7]:
def string_processing(string):
    symbols = ['%', '&', '(', ')', '*', '+', '/', '[', '\\', ']', '`',
               '{', '}', '~', '«', '»','’','“','”','•','…','№','\ufeff', '\xa0', '=', '^']
    for s in symbols:
        string = string.replace(s, '')
    
    string = string.replace('$', 's')
    string = string.replace('ü', 'у')
    string = string.replace('є', 'е')
    string = string.replace('і', 'i')
    string = string.replace('ї', 'i')
    string = string.replace('ң', 'н')
    string = string.replace('à', 'а')
    string = string.replace('ß', 'b')
    string = string.replace('ó', 'о')
    string = string.replace('ù', 'и')
    
    string = string.replace('–', '-')
    string = string.replace('—', '-')
    string = string.replace('_', '-')
    string = string.replace('@', ' ')
    
    
    
    string = string.replace('.', ' ')
    string = string.replace('?', '')
    string = string.replace('!', '')
    string = string.replace(' - ', ' ')
    string = string.replace(' -', ' ')
    string = string.replace('- ', '')
    string = string.replace('"', '')

    string = string.replace(':', '')
    string = string.replace(';', '')
    string = string.replace('   ', ' ')
    string = string.replace('  ', ' ')
    
    
    string = string.strip()
    
    if string.endswith(','):
        string = string[:-1]
        
    if string.endswith('-'):
        string = string[:-1]        
    
    return string

In [8]:
## чистка
texts_ = []
rapers_ = []
i = 0
for txt in tqdm(texts):
    raper = rapers[i]
    i += 1
    # проверка на наличие маркеров белых строк
    if txt.count('<w>') == 0:
        continue
    
    tmp = []    
    for string in txt:
        
        string = string_processing(string)
        
        if string == '':
            continue
        if string == ' ':
            continue
        if string == '  ':
            continue
        if string.startswith('[') and string.endswith(']'):
            continue
            
        if len(string) <= 5 and string != '<w>': #########
            continue
#         if re.match('^[0-9][.: ]', string):
#             continue            
        
        if string.count('припев') > 0:
            continue
        if string.count('куплет') > 0:
            continue
    
        tmp.append(string)

    texts_.append(tmp)
    rapers_.append(raper)


print(len(texts_))
print(len(rapers_))                


5156
5156


In [9]:
texts_[0][:5]

['мой бит качает',
 'он никого не напрягает',
 '<w>',
 'мой микрофон включён, мой ди-джей на месте',
 'приготовьтесь к уроку и все вместе']

In [10]:
for txt in tqdm(texts_):
    txt.append('<w>')

In [11]:
## определение ритмической структуры песни
structures = []


for txt in tqdm(texts_):
    
    struct = []
    for i in range(len(txt)):
        if txt[i] == '<w>': struct.append(i)
    structures.append(struct)        

In [12]:
structures[:10]

[[2, 19, 22, 39, 42, 59, 62],
 [32, 37, 42, 63, 68, 73],
 [15, 20, 37, 42, 59, 64],
 [1, 18, 23, 32, 42, 59, 73, 77],
 [33, 38, 43, 69, 74, 79, 132, 134, 139, 144],
 [9, 36, 46, 65, 75, 85],
 [19, 24, 41, 46, 61, 66],
 [1, 18, 35, 52, 70, 72, 90, 110],
 [2, 17, 20, 37, 44, 47],
 [20, 25, 42, 47, 64, 69, 70, 75]]

In [13]:
texts_[7][:5]

['а напоследок я бухну, ха-ха-ха',
 '<w>',
 'вечереет',
 'солнце скидывает яркую одежду',
 'на безбрежном небосводе облака шептаться стали нежно']

In [14]:
rapers_[-1]

'Schokk'

In [15]:
print(len(structures))
print(len(texts_))

5156
5156


In [16]:
strof = []
author = []

for struct in tqdm(structures):
    index = structures.index(struct)
    raper = rapers_[index]
    txt = texts_[index]
    
    start = 0
    for next_ in struct:
        strof_count = next_ - start - 1
#         # двустишия
#         if strof_count == 2:
#             s = []
#             s.append(txt[start+1])
#             s.append(txt[start+2])
#             s.extend(s)
#             if not s in strof:
#                 strof.append(s)
#                 author.append(raper)
#             start = next_
            
        # четверостишия
        if (strof_count) % 4 == 0 and strof_count > 0:
            for i in range(start, start + strof_count, 4): # цикл по четверостишиям
                s = []
                for j in range(i+1, i+5, 1): # цикл по строкам в четверостишии
                    s.append(txt[j])
                if not s in strof and s[0] != s[1] and s[0] != s[2] and s[0] != s[3] :
                    # print(strof_count, s) 
                    strof.append(s)
                    author.append(raper)
            start = next_
        else:
            start = next_
            continue        
    
    
    
print(len(strof))
print(len(author))                


26205
26205


In [17]:
strof[:100]

[['мой микрофон включён, мой ди-джей на месте',
  'приготовьтесь к уроку и все вместе',
  'поднимите руки вверх, когда я толкаю тему',
  'теорему о том, как атаковать по биту смело'],
 ['мои рифмы оставляют шрамы на сердцах',
  'наводят страх, я вижу, ты в моих руках',
  'ведь я всегда говорю правду, даже когда вру',
  'я врождённый теоретик и буду им, пока дышу'],
 ['пока не умру, не обрету покой на том свете',
  'словно дети в моём уме бушует буйный ветер',
  'мои темы бесконечный экспресс, злой пресс',
  'для всех фальшивых мс вечный стресс'],
 ['в моих строках виден блеск, как в волнах плеск эмоций',
  'ненависть к врагам, как революция в камбодже',
  'мои люди не хотят просто так спокойно жить',
  'быть, как пугливое стадо, а, как солдаты, всё бомбить'],
 ['жизнь, как продажное дерьмо, суть ничто',
  'мне плевать на всё, я буду жить, как суждено',
  'я верю только богу, мою дорогу',
  'освещает звезда, что выше разума, дороже слова'],
 ['сил очень мало, но моя вера велика',
  'она

In [18]:
s0 = [s[0] for s in strof]
s1 = [s[1] for s in strof]
s2 = [s[2] for s in strof]
s3 = [s[3] for s in strof]

In [19]:
lens = []
for s in strof:
    l = len(s[0]) + len(s[1]) + len(s[2]) + len(s[3])
    l = l/4.
    lens.append(l)

In [20]:
type(lens[0])

float

In [21]:
d = {'raper':author,'s0':s0, 's1':s1, 's2':s2, 's3':s3, 'len': lens}
df = pd.DataFrame(d)
df.len = df.len.astype(float)
df = df.sort_values(by='len')
df.head()

,raper,s0,s1,s2,s3,len
24796,Триада,шикарно,эрмитаж,показ мод,зимний,7.25
7917,Бонч Бру Бонч,тарантул,тик-так,тик-так,тик-так,7.25
12852,SixtyNine,это война,она все,она все,она все,7.50
12398,Смоки Мо (Некий Молла),еле заметным,не лишним,поверь,истина,8.25
12605,Рем Дигга,без души,без имен,не приком,не причем,8.50


In [22]:
df.head(200)

,raper,s0,s1,s2,s3,len
24796,Триада,шикарно,эрмитаж,показ мод,зимний,7.25
7917,Бонч Бру Бонч,тарантул,тик-так,тик-так,тик-так,7.25
12852,SixtyNine,это война,она все,она все,она все,7.50
12398,Смоки Мо (Некий Молла),еле заметным,не лишним,поверь,истина,8.25
12605,Рем Дигга,без души,без имен,не приком,не причем,8.50
3196,Капа,бутылка,фольга,иголка,в пипетке пятка,8.50
24055,Babangida (Бабангида),взял плюшку,покурил,тебя плющит,крапали,9.00
24794,Триада,пикники,секреты,интриги,с мамой в каннах,9.25
15376,Триада,спускает курки,в таких,как я и ты,я и ты,9.25
12604,Рем Дигга,просто сон,вечный сон,стон сирен,в унисон,9.50


In [23]:
df.shape

(26205, 6)

In [24]:
print(np.sum(df.s0.isnull()))
print(np.sum(df.s1.isnull()))
print(np.sum(df.s2.isnull()))
print(np.sum(df.s3.isnull()))

0
0
0
0


In [25]:
df.dtypes

raper     object
s0        object
s1        object
s2        object
s3        object
len      float64
dtype: object

In [26]:
df = df[df.s0!='']
print(df.shape)
df = df[df.s1!='']
print(df.shape)
df = df[df.s2!='']
print(df.shape)
df = df[df.s3!='']
print(df.shape)

(26205, 6)
(26205, 6)
(26205, 6)
(26205, 6)


In [27]:
all_text = list(df.s3.values)
all_text = ' '.join(all_text)

In [28]:
all_text[:50]

'зимний тик-так она все истина не причем в пипетке '

In [29]:
chars = set(all_text)

In [30]:
chars

{' ',
 "'",
 ',',
 '-',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 'а',
 'б',
 'в',
 'г',
 'д',
 'е',
 'ж',
 'з',
 'и',
 'й',
 'к',
 'л',
 'м',
 'н',
 'о',
 'п',
 'р',
 'с',
 'т',
 'у',
 'ф',
 'х',
 'ц',
 'ч',
 'ш',
 'щ',
 'ъ',
 'ы',
 'ь',
 'э',
 'ю',
 'я',
 'ё'}

In [31]:
df.to_csv('data/rap.csv', sep=';', index=False)